Эта записная книжка проведет вас через шаги, необходимые для обучения базовой модели деревьев с градиентным усилением с использованием леса принятия решений TensorFlow на наборе данных «Успеваемость учащихся из игры», доступном для этого соревнования, чтобы предсказать, будут ли игроки правильно отвечать на вопросы. Мы будем загружать данные из файла CSV. Грубо код будет выглядеть так:

In [ ]:
import tensorflow_decision_forests as tfdf
import pandas as pd

dataset = pd.read_csv("project/dataset.csv")
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(dataset, label="my_label")

model = tfdf.keras.GradientBoostedTreesModel()
model.fit(tf_dataset)

print(model.summary())

Мы также научимся оптимизировать чтение больших наборов данных, проработаем некоторые функции, визуализируем данные и рассчитываем лучшие результаты с помощью F1-показателя.

Леса принятия решений — это семейство древовидных моделей, включая случайные леса и деревья с градиентным усилением. Это лучшее место для начала работы с табличными данными, и они часто превосходят (или обеспечивают надежный базовый уровень) до того, как вы начнете экспериментировать с нейронными сетями.

Один из ключевых аспектов TensorFlow Decision Forests, который делает его еще более подходящим для этого соревнования, особенно с учетом ограничений времени выполнения, заключается в том, что он был тщательно протестирован для обучения и вывода на процессорах, что позволяет обучать его на младших машинах. .

# Import the Required Libraries

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_decision_forests as tfdf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("TensorFlow Decision Forests v" + tfdf.__version__)
print("TensorFlow Addons v" + tfa.__version__)
print("TensorFlow v" + tf.__version__)

# Load the Dataset

Поскольку набор данных огромен, некоторые люди могут столкнуться с ошибками памяти при чтении набора данных из csv. Чтобы избежать этого, мы попытаемся оптимизировать память, используемую Pandas для загрузки и хранения набора данных.

Когда Pandas загружает набор данных, по умолчанию он автоматически определяет типы данных различных столбцов. Независимо от максимального значения, хранящегося в этих столбцах, Pandas присваивает int64 для числовых столбцов, float64 для столбцов с плавающей запятой, объект dtype для строковых столбцов и т. д.

Мы можем уменьшить размер этих столбцов в памяти, понизив числовые столбцы до меньших типов (например, int8, int32, float32 и т. д.), если их максимальные значения не нуждаются в больших типах для хранения (например, int64, float64). и т. д.).

Точно так же Pandas автоматически определяет строковые столбцы как тип данных объекта. Чтобы уменьшить использование памяти строковыми столбцами, в которых хранятся категориальные данные, мы указываем их тип данных как категорию.

Многие столбцы в этом наборе данных могут быть преобразованы в меньшие типы.

Мы предоставим пандам набор dtypes для столбцов при чтении набора данных.

In [ ]:
# Reference: https://www.kaggle.com/competitions/predict-student-performance-from-game-play/discussion/384359
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}

dataset_df = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv', dtype=dtypes)
print("Full train dataset shape is {}".format(dataset_df.shape))

Данные состоят из 20 столбцов и 26296946 записей. Мы можем увидеть все 20 измерений нашего набора данных, распечатав первые 5 записей, используя следующий код:

In [ ]:
# Display the first 5 examples
dataset_df.head(5)

Обратите внимание, что session_id однозначно идентифицирует сеанс пользователя.

Load the labels

Метки для обучающего набора данных хранятся в файле train_labels.csv. Он состоит из информации о том, правильно ли ответил пользователь в конкретном сеансе на каждый вопрос. Загрузите данные меток, выполнив следующий код.

In [ ]:
labels = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')

Каждое значение в столбце session_id представляет собой комбинацию сеанса и номера вопроса. Мы разделим их на отдельные столбцы для удобства использования.

In [ ]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

Давайте посмотрим на первые 5 записей меток, используя следующий код:

In [ ]:
# Display the first 5 examples
labels.head(5)

Наша цель — обучить модели для каждого вопроса, чтобы правильно предсказать метку для любого входного пользовательского сеанса.

# Bar chart for label column: correct
# Гистограмма для столбца меток: верно

Сначала мы построим гистограмму для значений метки правильно.


In [ ]:
plt.figure(figsize=(3, 3))
plot_df = labels.correct.value_counts()
plot_df.plot(kind="bar", color=['b', 'c'])

Теперь давайте нанесем на график значения столбца метки, правильные для каждого вопроса.

In [ ]:
plt.figure(figsize=(10, 20))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle("\"Correct\" column values for each question", fontsize=14, y=0.94)
for n in range(1,19):
    #print(n, str(n))
    ax = plt.subplot(6, 3, n)

    # filter df and plot ticker on the new subplot axis
    plot_df = labels.loc[labels.q == n]
    plot_df = plot_df.correct.value_counts()
    plot_df.plot(ax=ax, kind="bar", color=['b', 'c'])
    
    # chart formatting
    ax.set_title("Question " + str(n))
    ax.set_xlabel("")

Подготовьте набор данных

Как указано в обзоре конкурса, набор данных представляет нам вопросы и данные в порядке уровней — сегментов уровней (представленных столбцом level_group) 0–4, 5–12 и 13–22. Мы должны предсказать правильность вопросов каждого сегмента по мере их представления. Для этого мы создадим основные агрегатные функции из соответствующих столбцов. Вы можете создать больше функций, чтобы повысить свои баллы.

Во-первых, мы создадим два отдельных списка с именами категориальных столбцов и числовых столбцов. Мы будем избегать столбцов fullscreen, hq и music, поскольку они не добавляют никакой полезной ценности для этой постановки задачи.

In [ ]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y', 
        'screen_coor_x', 'screen_coor_y', 'hover_duration']

Для каждого категориального столбца мы сначала сгруппируем набор данных по session_id и level_group. Затем мы подсчитаем количество различных элементов в столбце для каждой группы и временно сохраним его.

Для всех числовых столбцов мы сгруппируем набор данных по идентификатору сеанса и level_group. Вместо подсчета количества отдельных элементов мы вычислим среднее значение и стандартное отклонение числового столбца для каждой группы и временно сохраним его.

После этого мы объединим временные фреймы данных, которые мы создали на предыдущем шаге, для каждого столбца, чтобы создать наш новый набор данных с инженерными функциями.

In [ ]:
# Reference: https://www.kaggle.com/code/cdeotte/random-forest-baseline-0-664/notebook

def feature_engineer(dataset_df):
    dfs = []
    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df

In [ ]:
dataset_df = feature_engineer(dataset_df)
print("Full prepared dataset shape is {}".format(dataset_df.shape))

Наш специально спроектированный набор данных состоит из 22 столбцов и 70686 записей.

# Basic exploration of the prepared dataset
# Базовое исследование подготовленного набора данных

Давайте распечатаем первые 5 записей, используя следующий код

In [ ]:
# Display the first 5 examples
dataset_df.head(5)

In [ ]:
dataset_df.describe()

# Numerical data distribution
# Распределение числовых данных

Давайте нанесем несколько числовых столбцов и их значение для каждой группы level_group:

In [ ]:
figure, axis = plt.subplots(3, 2, figsize=(10, 10))

for name, data in dataset_df.groupby('level_group'):
    axis[0, 0].plot(range(1, len(data['room_coor_x_std'])+1), data['room_coor_x_std'], label=name)
    axis[0, 1].plot(range(1, len(data['room_coor_y_std'])+1), data['room_coor_y_std'], label=name)
    axis[1, 0].plot(range(1, len(data['screen_coor_x_std'])+1), data['screen_coor_x_std'], label=name)
    axis[1, 1].plot(range(1, len(data['screen_coor_y_std'])+1), data['screen_coor_y_std'], label=name)
    axis[2, 0].plot(range(1, len(data['hover_duration'])+1), data['hover_duration_std'], label=name)
    axis[2, 1].plot(range(1, len(data['elapsed_time_std'])+1), data['elapsed_time_std'], label=name)
    

axis[0, 0].set_title('room_coor_x')
axis[0, 1].set_title('room_coor_y')
axis[1, 0].set_title('screen_coor_x')
axis[1, 1].set_title('screen_coor_y')
axis[2, 0].set_title('hover_duration')
axis[2, 1].set_title('elapsed_time_std')

for i in range(3):
    axis[i, 0].legend()
    axis[i, 1].legend()

plt.show()

Теперь давайте разделим набор данных на наборы данных для обучения и тестирования:


In [ ]:
def split_dataset(dataset, test_ratio=0.20):
    USER_LIST = dataset.index.unique()
    split = int(len(USER_LIST) * (1 - 0.20))
    return dataset.loc[USER_LIST[:split]], dataset.loc[USER_LIST[split:]]

train_x, valid_x = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_x), len(valid_x)))

# Select a Model
# Выберите модель

Есть несколько моделей на основе дерева на ваш выбор.

- RandomForestModel
- GradientBoostedTreesModel
- CartModel
- DistributedGradientBoostedTreesModel
Мы можем перечислить все доступные модели в TensorFlow Decision Forests, используя следующий код:

In [ ]:
tfdf.keras.get_all_models()

Для начала мы будем работать с моделью деревьев с усилением градиента. Это один из известных алгоритмов обучения Decision Forest.

Дерево решений с градиентным усилением представляет собой набор неглубоких деревьев решений, обучаемых последовательно. Каждое дерево обучается предсказывать, а затем «исправлять» ошибки ранее обученных деревьев.

# How can I configure a tree-based model?
# Как настроить древовидную модель?

TensorFlow Decision Forests предоставляет вам хорошие значения по умолчанию (например, самые высокие гиперпараметры в наших тестах, слегка измененные для запуска в разумные сроки). Если вы хотите настроить алгоритм обучения, вы найдете множество опций, которые вы можете изучить, чтобы получить максимально возможную точность.

Вы можете выбрать шаблон и/или установить параметры следующим образом:

In [ ]:
rf = tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1")

You can read more here.

# Training

Мы будем обучать модель для каждого вопроса, чтобы предсказать, будет ли пользователь правильно отвечать на вопрос. Всего в наборе данных 18 вопросов. Следовательно, мы будем обучать 18 моделей, по одной на каждый вопрос.

Нам нужно предоставить несколько структур данных для нашего цикла обучения, чтобы хранить обученные модели, прогнозы для набора проверки и оценки для обученных моделей.

Мы создадим их, используя следующий код:

In [ ]:
# Получить уникальный список пользовательских сеансов в наборе данных проверки. Мы назначили
# `session_id` в качестве индекса нашего набора данных с инженерными функциями. Отсюда получение
# уникальные значения в столбце index дадут нам список пользователей в
# набор для проверки.
VALID_USER_LIST = valid_x.index.unique()

# Создать фрейм данных для хранения прогнозов каждого вопроса для всех пользователей
# в проверочном наборе.
# Для этого требуется размер фрейма данных:
# (количество: пользователей в наборе проверки x количество вопросов).
# Мы инициализируем все предсказанные значения во фрейме данных нулем.
# Столбец индекса фрейма данных - это пользовательский `session_id`.
prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)

# Создайте пустой словарь для хранения моделей, созданных для каждого вопроса.
models = {}

# Создайте пустой словарь для хранения оценки каждого вопроса.
evaluation_dict ={}

Перед обучением данных мы должны понять, как level_groups и вопросы связаны друг с другом.

В этой игре первая контрольная точка викторины (т.е. вопросы с 1 по 3) наступает после прохождения уровней с 0 по 4. Таким образом, для обучающих вопросов с 1 по 3 мы будем использовать данные из level_group 0-4. Точно так же мы будем использовать данные из level_group 5-12 для обучения вопросам с 4 по 13 и данные из level_group 13-22 для обучения вопросам с 14 по 18.

Мы будем обучать модель для каждого вопроса и хранить обученную модель в словаре моделей.

In [ ]:
# Повторить вопросы с 1 по 18, чтобы обучить модели для каждого вопроса, оценить
# обученную модель и сохранить предсказанные значения.
for q_no in range(1,19):

    # Выберите группу уровня для вопроса на основе q_no.
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    print("### q_no", q_no, "grp", grp)
    
        
    # Отфильтровать строки в наборах данных на основе выбранной группы уровней.
    train_df = train_x.loc[train_x.level_group == grp]
    train_users = train_df.index.values
    valid_df = valid_x.loc[valid_x.level_group == grp]
    valid_users = valid_df.index.values

    # Выберите метки для соответствующего q_no.
    train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
    valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]

    # Добавляем метку к отфильтрованным наборам данных.
    train_df["correct"] = train_labels["correct"]
    valid_df["correct"] = valid_labels["correct"]

    # Прежде чем мы сможем обучить модель, необходимо выполнить еще один шаг.
     # Нам нужно преобразовать набор данных из формата Pandas (pd.DataFrame)
     # в формат наборов данных TensorFlow (tf.data.Dataset).
     # TensorFlow Datasets — это высокопроизводительная библиотека для загрузки данных.
     # что полезно при обучении нейронных сетей с помощью ускорителей, таких как GPU и TPU.
     # Мы опускаем `level_group`, так как он больше не нужен для обучения.
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df.loc[:, train_df.columns != 'level_group'], label="correct")
    valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_df.loc[:, valid_df.columns != 'level_group'], label="correct")

    # Теперь мы создадим модель деревьев с усилением градиента с настройками по умолчанию.
     # По умолчанию модель настроена на обучение задаче классификации.
    gbtm = tfdf.keras.GradientBoostedTreesModel(verbose=0)
    gbtm.compile(metrics=["accuracy"])

    # Обучить модель.
    gbtm.fit(x=train_ds)

    # Сохраняем модель
    models[f'{grp}_{q_no}'] = gbtm

    # Оцените обученную модель в наборе данных проверки и сохраните
     # точность оценки в `evaluation_dict`.
    inspector = gbtm.make_inspector()
    inspector.evaluation()
    evaluation = gbtm.evaluate(x=valid_ds,return_dict=True)
    evaluation_dict[q_no] = evaluation["accuracy"]         

    # Используйте обученную модель, чтобы делать прогнозы по набору данных проверки и
     # сохранить предсказанные значения в кадре данных `prediction_df`.
    predict = gbtm.predict(x=valid_ds)
    prediction_df.loc[valid_users, q_no-1] = predict.flatten()     

# Inspect the Accuracy of the models.¶


Мы обучили модель для каждого вопроса. Теперь давайте проверим точность каждой модели и общую точность всех моделей вместе взятых.

Примечание. Поскольку распределение меток несбалансировано, мы не можем делать предположения о производительности модели только на основе показателя точности.

In [ ]:
for name, value in evaluation_dict.items():
  print(f"question {name}: accuracy {value:.4f}")

print("\nAverage accuracy", sum(evaluation_dict.values())/18)

# Visualize the model

Одним из преимуществ древовидных моделей является то, что мы можем легко их визуализировать. По умолчанию количество деревьев, используемых в случайных лесах, равно 300.

Давайте выберем одну модель из списка моделей и выберем дерево для отображения ниже.

In [ ]:
tfdf.model_plotter.plot_model_in_colab(models['0-4_1'], tree_idx=0, max_depth=3)

# Variable importances

Переменная важность обычно указывает, насколько функция влияет на прогнозы или качество модели. Существует несколько способов определения важных функций с помощью TensorFlow Decision Forest. Давайте выберем одну модель из списка моделей и проверим ее.

Давайте перечислим доступные значения переменных для деревьев решений:

In [ ]:
inspector = models['0-4_1'].make_inspector()

print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)

В качестве примера давайте отобразим важные функции для важности переменной NUM_AS_ROOT.

Чем больше показатель важности для NUM_AS_ROOT, тем большее влияние он оказывает на результат модели для вопроса 1 (т. е. модель["0-4_1"]).

По умолчанию список отсортирован от наиболее важного к наименее важному. Из вывода вы можете сделать вывод, что функция в верхней части списка используется в качестве корневого узла в большинстве деревьев в деревьях с градиентным усилением, чем любая другая функция.

In [ ]:
# Each line is: (feature name, (index of the feature), importance score)
inspector.variable_importances()["NUM_AS_ROOT"]

# Threshold-Moving for Imbalanced Classification
# Изменение порога для несбалансированной классификации

Поскольку значения правильного столбца довольно несбалансированы, использование порога по умолчанию 0,5 для сопоставления прогнозов с классами 0 или 1 может привести к снижению производительности. В таких случаях, чтобы улучшить производительность, мы вычисляем балл F1 для определенного диапазона пороговых значений и пытаемся найти лучший порог, также известный как порог с наивысшим баллом F1. Затем мы будем использовать этот порог для сопоставления предсказанных вероятностей с метками классов 0 или 1.

Обратите внимание, что мы используем показатель F1, так как это лучший показатель, чем точность, для оценки проблем с дисбалансом классов.

In [ ]:
# Создадим фрейм данных нужного размера:
# (количество: пользователей в проверочном наборе x количество: вопросов) инициализированы нулевыми значениями
# для хранения истинных значений метки `correct`.
true_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)
for i in range(18):
    # Получить истинные метки.
    tmp = labels.loc[labels.q == i+1].set_index('session').loc[VALID_USER_LIST]
    true_df[i] = tmp.correct.values

max_score = 0; best_threshold = 0

# Прокрутите пороговые значения от 0,4 до 0,8 и выберите порог с помощью
# самый высокий балл F1.
for threshold in np.arange(0.4,0.8,0.01):
    metric = tfa.metrics.F1Score(num_classes=2,average="macro",threshold=threshold)
    y_true = tf.one_hot(true_df.values.reshape((-1)), depth=2)
    y_pred = tf.one_hot((prediction_df.values.reshape((-1))>threshold).astype('int'), depth=2)
    metric.update_state(y_true, y_pred)
    f1_score = metric.result().numpy()
    if f1_score > max_score:
        max_score = f1_score
        best_threshold = threshold
        
print("Best threshold ", best_threshold, "\tF1 score ", max_score)

# Submission

Представление Здесь вы будете использовать расчет best_threshold в предыдущей ячейке.

In [ ]:
# Reference
# https://www.kaggle.com/code/philculliton/basic-submission-demo
# https://www.kaggle.com/code/cdeotte/random-forest-baseline-0-664/notebook


import jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()

limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}

for (test, sample_submission) in iter_test:
    test_df = feature_engineer(test)
    grp = test_df.level_group.values[0]
    a,b = limits[grp]
    for t in range(a,b):
        gbtm = models[f'{grp}_{t}']
        test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df.loc[:, test_df.columns != 'level_group'])
        predictions = gbtm.predict(test_ds)
        mask = sample_submission.session_id.str.contains(f'q{t}')
        n_predictions = (predictions > best_threshold).astype(int)
        sample_submission.loc[mask,'correct'] = n_predictions.flatten()
    
    env.predict(sample_submission)

In [ ]:
! head submission.csv